## 语言模型简介
假设一门语言中所有可能的句子服从某一概率分布，每个句子出现的概率加起来为1，那么"语言模型"的任务就是预测每个句子在语言中出现的概率。对于语言中常见的句子，一个好的语言模型应得出相对较高的概率，而不合语法的句子，计算出来的概率则应接近于零。语言模型仅仅对句子出现的概率进行建模，并不去理解句子的内容含义。<br/>
首先一个句子可以被看成是一个单词序列：
$$S=(w_1,w_2,w_3,w_4,\cdots, w_m)$$
其中m为句子的长度。那么，它的概率可以表示为：
$$
\begin{aligned}
p(S) &= p(w_1, w_2, \cdots, w_m) \\
&= p(w_1)p(w2|w_1)p(w_3|w1,w_2)\cdots p(w_m|w_1,w_2,\cdots ,w_{m-1})
\end{aligned}
$$
其中$p(w_m|w_1,w_2,\cdots ,w_{m-1})$表示已知前m-1个单词时，第m个单词为$w_m$的条件概率。如果能对这一项建模，那么只要把每个位置的条件概率相乘，就能计算一个句子出现的概率。然而一般来说，任何一门语言的词汇量很大，词组的组合不计其数，一般的句子长度远远超出了实际可行范围。为了估计这些参数，常见的方法有n-gram模型、决策树、条件随机场、最大熵模型、神经网络语言模型等。<br/>
以n-gram模型为例，为了控制参数数量，n-gram模型做了一个假设：当前单词的出现概率仅仅与前面的n-1个单词相关，因此以上公式可以近似为：
$$
p(S) = p(w_1, w_2, \cdots, w_m) = \prod_i^m p(w_i|w_{i-n+1},\cdots , w_{i-1})
$$
n-gram模型里的n指的是当前单词依赖它前面的单词的个数。通常n可取1、2、3、4，其中n取1、2、3时分别称为unigram、bigram和trigram。当n越大时，n-gram模型理论上越精确，但也越复杂，需要的计算量和训练语料数据量也越大，因此n≥4的情况很少。<br/>
n-gram模型的参数一般采用最大似然估计(Maximum Likelihood Estimation，MLE)方法计算：
$$
p(w_i|w_{i-n+1}, \cdots, w_{i-1}) = \frac{C(w_{i-n+1}, \cdots, w_{i-1}, w_i)}{C(w_{i-n+1}, \cdots, w_{i-1})}
$$
其中$C(X)$表示单词序列X在训练语料中出现的次数。训练语料的规模越大，参数估计的结果越可靠。但是即使训练数据的规模非常大时，还是有很多单词序列在训练语料中不会出现，这就会导致很多参数为0。为了避免这种情况的出现导致整个概率为0，使用MLE方法时需要加入平滑避免参数取值为0。
## 语言模型的评价方法
语言模型效果好坏最常用评价指标是复杂度(perplexity)，它刻画的是语言模型预测一个语言样本的能力。在测试集上得到的perplexity越低，说明建模效果越好。计算perplexity值的公式：
$$
\begin{aligned}
perplexity(S) &= p(w_1, w_2,\cdots , w_m)^{-1/m} \\
&= \sqrt[m]{\frac{1}{p(w_1, w_2,\cdots , w_m)}} \\
& = \sqrt[m]{\prod_{i=1}^{m}\frac{1}{p(w_i|w_1,\cdots ,w_{i-1})}}
\end{aligned}
$$
可以看出，perplexity实际是计算每一个单词得到的概率的倒数的几何平均。因此perplexity可以理解为平均分支系数(average branching factor)，即模型预测下一个词时的平均可选择数量。<br/>
目前PTB(Penn Tree Bank)数据集上最好的语言模型perplexity为47.7，也就是说，平均情况下，该模型预测下一个词时，有47.7个词等可能地可以作为下一个词的合理选择。<br/>
在语言模型的训练中，通常采用perplexity的对数表达形式：
$$\log(perplexity(S)) = -\frac{1}{m} \sum_{i=1}^{m}\log p(w_i|w_1,\cdots ,w_{i-1})$$
相比乘积求平凡根的方式，使用加法的形式可以加速计算，同时避免概率乘积数值过小而导致浮点数向下溢出的问题。<br/>
在数学上，log perplexity可以看出真实分布与预测分布之间的交叉熵(Cross Entropy)。交叉熵描述了两个概率分布之间的一种距离。假设$x$是离散变量，$u(x)$和$v(x)$是两个与x相关的概率分布，那么$u$和$v$之间的交叉熵的定义是在分布u下$-\log(v(x))$的期望值：
$$
H(u,v) = E_{u}[-log v(x)] = -\sum_{x} u(x)\log v(x)
$$
把x看成单词，$u(x)$为每个位置上单词的真实分布，$v(x)$为模型的预测分布$p(w_i|w_1,\cdots, w_{i-1})$，就可以看出log perplexity和交叉熵是等价的。唯一区别在于有语言的真实分布式未知的，因此在log perplexity的定义中，真实分布用测试语料中的取样代替，语料中出现的单词$w_i$的概率为1，出现其他单词的概率均为0。
$$
\begin{aligned}
&u(x|w_1, \cdots, w_{i-1}) = \begin{cases} 1,  & x=w_i \\ 0, & x \neq w_i 
\end{cases} \\
H(u, v) &= -\sum_{x}u(x)\log v(x) \\
&= -\frac{1}{m} \sum_{i=1}^{m}(\sum_{x} u(x|w_1,\cdots,w_{i-1})\log p(x|w_1, \cdots, w_{i-1})) \\
&= -\frac{1}{m} \sum_{i=1}^{m}(1.0\times \log p(x|w_1, \cdots, w_{i-1}) + \sum_{x\neq w_i} 0.0\times p(x|w_1, \cdots, w_{i-1})) \\
&= -\frac{1}{m} \sum_{i=1}^{m}\log p(x|w_1, \cdots, w_{i-1}) \\
&= \log(perplexity(S))
\end{aligned}
$$